# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [47]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [48]:
!cat talks.tsv

title	type	category	collection	url_slug	venue	date	location	talk_url	place	conf
On distinguishing digraphs by their quasisymmetric B-polynomial	Talk	talks	talks	In-house-1	Dept of Mathematics, IIT Madras	2022-07-01	Chennai, India		Chennai, India	In-House Symposium 2022
A counterexample to Hedetnieme’s Conjecture	Talk	talks	talks	mu-1	Mumbai University	2020-02-01	Mumbai, India		Mumbai, India	Invited Talk
On distinguishing digraphs by their quasisymmetric B-polynomial	Talk	talks	talks	gustave	Gustave-Eiffel University	2023-04-01	Champs-Sur-Marne, France		Paris, France	Algebraic Combinatorics Seminar
On digraph analog of Stanley’s Tree Conjecture	Talk	talks	talks	irif	IRIF	2023-04-04	Paris, France		Paris, France	Graph Theory Seminar
Stanley’s Tree Isomorphism Conjecture	Talk	talks	talks	In-house-2	Dept of Mathematics, IIT Madras	2023-07-01	Chennai, India		Chennai, India	In-House Symposium 2023
On distinguishing proper q-caterpillars by their chromatic symmetric functions	Talk	talks	talks	

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [49]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,category,collection,url_slug,venue,date,location,talk_url,place,conf
0,On distinguishing digraphs by their quasisymme...,Talk,talks,talks,In-house-1,"Dept of Mathematics, IIT Madras",2022-07-01,"Chennai, India",NaN,"Chennai, India",In-House Symposium 2022
1,A counterexample to Hedetnieme’s Conjecture,Talk,talks,talks,mu-1,Mumbai University,2020-02-01,"Mumbai, India",NaN,"Mumbai, India",Invited Talk
2,On distinguishing digraphs by their quasisymme...,Talk,talks,talks,gustave,Gustave-Eiffel University,2023-04-01,"Champs-Sur-Marne, France",NaN,"Paris, France",Algebraic Combinatorics Seminar
3,On digraph analog of Stanley’s Tree Conjecture,Talk,talks,talks,irif,IRIF,2023-04-04,"Paris, France",NaN,"Paris, France",Graph Theory Seminar
4,Stanley’s Tree Isomorphism Conjecture,Talk,talks,talks,In-house-2,"Dept of Mathematics, IIT Madras",2023-07-01,"Chennai, India",NaN,"Chennai, India",In-House Symposium 2023
5,On distinguishing proper q-caterpillars by the...,Talk,talks,talks,imsc,Institute of Mathematical Sciences,2023-09-01,"Adyar, India",NaN,"Chennai, India",Algebraic Combinatorics Seminar
6,Essentials of Mathematics in competitive exams,Talk,expository,expository,ruia,R. N. Ruia College,2020-11-01,"Matunga, India",NaN,"Mumbai, India",Invited Talk
7,Linear algebra methods in combinatorics,Talk,expository,expository,Sum-talk,"Dept of Mathematics, IIT Madras",2023-06-01,"Chennai, India",NaN,"Chennai, India",Summer School-2023
8,Polynomial factorizations in Integral Domains,Talk,expository,expository,Sum-talk2,"Dept of Mathematics, IIT Madras",2024-06-01,"Chennai, India",NaN,"Chennai, India",Summer School-2024


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [50]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [51]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.date)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
    
    if len(str(item.category)) > 3:
        md += 'category: "' + str(item.category) + '"\n'
    
    if len(str(item.place)) > 3:
        md += 'place: "' + str(item.place) + '"\n'
    
    if len(str(item.conf)) > 3:
        md += 'name: "' + str(item.conf) + '"\n'

    
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [52]:
!ls ../_talks

2020-02-01-mu-1.md	  2023-07-01-In-house-2.md  paderborn.md
2020-11-01-ruia.md	  2023-09-01-imsc.md	    rms.md
2022-07-01-In-house-1.md  2024-06-01-Sum-talk2.md   talkmap.ipynb
2023-04-01-gustave.md	  icgta.md		    talkmap.py
2023-04-04-irif.md	  meru-1.md		    talks.ipynb
2023-06-01-Sum-talk.md	  meru-2.md		    Untitled.ipynb


In [53]:
!cat ../_talks/2023-04-01-gustave.md

---
title: "On distinguishing digraphs by their quasisymmetric B-polynomial"
collection: talks
type: "Talk"
permalink: /talks/2023-04-01-gustave
venue: "Gustave-Eiffel University"
date: 2023-04-01
location: "Champs-Sur-Marne, France"
category: "talks"
place: "Paris, France"
name: "Algebraic Combinatorics Seminar"
---
